## Word2Vec

### Introdução

Em machine learning, o objetivo é empregar o poder computacional para resolver problemas do mundo real. Diferentemente dos humanos, que possuem percepção direta, os computadores operam fundamentalmente com números, convertidos em última instância para sequências de __0's__ e __1's__.

Atualmente, algoritmos processam diversos tipos de dados, incluindo números, imagens, áudios e textos. Para que um computador possa trabalhar com essas informações, é necessário encontrar maneiras de representá-las numericamente, permitindo sua manipulação. No contexto do texto, existem várias técnicas capazes de convertê-lo em informação numérica. Essas técnicas variam desde abordagens mais simples, como a contagem de palavras no modelo _Bag of Words_, até modelos mais sofisticados, como os _Transformers_. Este estudo se concentra na técnica __Word2Vec__.

O __Word2Vec__ consiste em representar palavras em um espaço vetorial de _n-dimensões_, de forma que essa representação capture seu significado semântico. Geralmente, palavras semanticamente relacionadas, como "mulher" e "garota", terão representações vetoriais próximas nesse espaço. Em contraste, a similaridade vetorial entre palavras menos relacionadas, como "mulher" e "homem", será menor. Essa lógica se aplica a outros exemplos de palavras como visualizado abaixo.
> Word2vec é um algoritmo para obter word embeddings treinando uma rede neural rasa (com apenas uma hidden layer) com duas arquiteturas possíveis: CBOW ou Skip-Gram. ([Word Embedding: fazendo o computador entender o significado das palavras](https://medium.com/turing-talks/word-embedding-fazendo-o-computador-entender-o-significado-das-palavras-92fe22745057))

![Representação Vetorial de Palavras em um Plano 3D](https://miro.medium.com/v2/resize:fit:868/0*Cgod6JuBcJyd9GVM)

### Arquiteturas do Word2Vec

Antes de iniciar a exploração, vou importar algumas bibliotecas necessárias que utilizarei para esse projeto (evitar importações desnecessárias ou repetitivas).

In [7]:
import os
import numpy as np
import pandas as pd

Além disso, vou criar algumas funções comuns que vou utilizar nesse notebook python

In [ ]:
import requests
import zipfile

# Funtion to download zip file from URL
def download_file_zip(URL: str, file_name: str):
    os.makedirs("data/model/zipfiles", exist_ok=True)

    try:
        # Faz o download
        response = requests.get(URL)
        response.raise_for_status()  # Lança erro se status != 200
        
        # Salva o arquivo
        file_path = os.path.join("data", "model", "zipfiles", f"{file_name}.zip")
        with open(file_path, "wb") as file:
            file.write(response.content)
        return f"data/model/zipfiles/{file_name}.zip"
    
    except Exception as e:
        print(f"Erro: {str(e)}")


def unzip_file(source_path: str, final_path: str):
    with zipfile.ZipFile(source_path, "r") as zip_ref:
        zip_ref.extractall(final_path)
        print("All unzip!")  # Substitua pelo caminho desejado

#### CBOW

A arquitetura CBOW (Continuous Bag-of-Words) tem como objetivo prever uma palavra central com base no contexto das palavras que a cercam. Ela utiliza mais processamento uma vez que ela necessita analisar as palavras ao redor para que ela busque qual melhor palavra que se encaixa naquele contexto.
Na palavra abaixo:
> Eu vou para a __________ estudar com a professora!

O algoritmo analisa as palavras ao redor da palavra buscada e por exemplo, identifica que estudar tem relação com outras palavras como __casa__, __escola__, __biblioteca,__ dentre outras haver com esse contexto. Quando ele analisa as outras palavras da frase ele vê que __professora__ está mais relacionado com __escola__ do que com __biblioteca__ por exemplo e com base nessa ideia ele indica de maneira probabilística que a melhor palavra indicada para esse caso seja __escola__.

Em geral, algumas __vantagens__ que acompanha esse tipo de técnica é que o CBOW converge mais rapidamente do que o Skip-gram, pois precisa prever uma única palavra central a partir de múltiplos contextos, o que torna o problema de aprendizado um pouco mais fácil. Além disso, ele apresenta boa representação para palavras frequentes já que CBOW tende a aprender boas representações para palavras que aparecem com frequência no corpus, pois se beneficia da agregação de informações de múltiplos contextos. Além disso, ele é menos sensível a palavras raras já que como ele utiliza o contexto para prever a palavra central, o impacto de palavras raras no treinamento geral pode ser menor.

##### Explorando um pouco a arquiterura CBOW

Baixando o modelo CBOW treinado para português brasileiro.

In [ ]:
URL_CBOW_100D = "http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s100.zip"

path_final = download_file_zip(URL_CBOW_100D, "cbow_ptbr_100d")
unzip_file(path_final, "data/model/cbow_ptbr_100d")

Download concluído: data\model\zipfiles\cbow_ptbr_100d.zip


In [ ]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('data/model/cbow_ptbr_100d/cbow_s100.txt')

### Referências

- [Word2Vec e sua importância na etapa de pré-processamento](https://medium.com/@everton.tomalok/word2vec-e-sua-importância-na-etapa-de-pré-processamento-d0813acfc8ab)
- [Word Embedding: fazendo o computador entender o significado das palavras](https://medium.com/turing-talks/word-embedding-fazendo-o-computador-entender-o-significado-das-palavras-92fe22745057)
- [What Is Word2Vec and How Does It Work?](https://swimm.io/learn/large-language-models/what-is-word2vec-and-how-does-it-work)